In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from datetime import datetime, date, timedelta
from typing import List, Tuple, Dict, Union

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tqdm import tqdm

sns.set()

In [ ]:
customers = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")

In [ ]:
articles = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")

In [ ]:
sample_submission = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv")

In [ ]:
sample_submission.to_dict("records")[0]

# Customer EDA

In [ ]:
display(
    customers.head(),
    f"[ num of customers: {len(customers)} ]",
    f"[ cols of customers: {list(customers.columns)}]",
)

In [ ]:
# inspect customer id

display(customers["customer_id"].value_counts())

In [ ]:
# What is FN?

display(
    customers["FN"].value_counts(),
    len(customers[customers["FN"] == 1.0]) / len(customers),
)

In [ ]:
# Num of Active Customers
display(
    customers["Active"].value_counts(),
    len(customers[customers["Active"] == 1.0]) / len(customers),
)

# 생각해볼점
- Active와 Dead 상태의 유저를 구분해서 작업해야할 필요가 있지 않을까?
- Active status가 0인 유저는 상품을 구매할 수 없고, 그렇다면 트렌드에 반영되지 않을수도 있다는 생각?

In [ ]:
# club_member_status
display(
    customers["club_member_status"].value_counts(),
    f"""Active Users Ratio: {
        len(customers[customers['club_member_status'] == 'ACTIVE']) / len(customers),
    }""",
    f"""PRE-CREATE Users Ratio: {
        len(customers[customers['club_member_status'] == 'PRE-CREATE']) / len(customers),
    }""",
    f"""LEFT-CLUB Users Ratio: {
        len(customers[customers['club_member_status'] == 'LEFT CLUB']) / len(customers),
    }""",
)

# Club Member에서 생각해볼점
- Active User만을 대상으로 모델링해도 괜찮을 것 같음
- Active, PRE-CREATE까지만 고려해서 모델링하자

In [ ]:
# fashion_news_frequency
customers["fashion_news_frequency"].value_counts()

In [ ]:
pd.DataFrame(customers["fashion_news_frequency"].value_counts()).plot(kind="bar")

In [ ]:
# Age
age_df = pd.DataFrame(customers["age"].value_counts()).reset_index()
age_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.figure(figsize=(20, 10))

start_range = 0
end_range = 20
sns.barplot(data=age_df[start_range:end_range], x="index", y="age", alpha=0.8, order=age_df[start_range:end_range]["index"].values)
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(20, 10))

start_range = 20
end_range = 40
sns.barplot(data=age_df[start_range:end_range], x="index", y="age", alpha=0.8, order=age_df[start_range:end_range]["index"].values)
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(20, 10))

start_range = 40
end_range = 60
sns.barplot(data=age_df[start_range:end_range], x="index", y="age", alpha=0.8, order=age_df[start_range:end_range]["index"].values)
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(20, 10))

start_range = 60
end_range = 100
sns.barplot(data=age_df[start_range:end_range], x="index", y="age", alpha=0.8, order=age_df[start_range:end_range]["index"].values)
plt.xticks(rotation=45)
plt.show()



In [ ]:
# postcal code
display(
    customers["postal_code"].value_counts(),
)

# 생각해볼점
- 사실 사용자마다의 주소가 제각각일텐데 어쨌든 이렇게 만 몇개씩 걸리는 거는 문제가 있다고 생각~
- 제대로 모델링할때 조사해봐야할 부분